In [1]:
!rm -rf ./results
!rm -rf ./logs

In [2]:
from tqdm import tqdm_notebook

In [3]:
from datasets import load_dataset

**prachathai67k**


In [4]:
prachathai67k = load_dataset("prachathai67k")

Reusing dataset prachathai67k (/root/.cache/huggingface/datasets/prachathai67k/prachathai67k/1.1.0/1737b80493191494f4069b2e9e320a31a2e3bea6de6c71047e4d9b281dffcdc2)


In [5]:
train_data = list(prachathai67k['train'].data['body_text'])+list(prachathai67k['test'].data['body_text'])

In [6]:
print(len(train_data))

61168


In [7]:
del prachathai67k

**thaisum**

In [8]:
thaisum = load_dataset("thaisum")

Reusing dataset thaisum (/root/.cache/huggingface/datasets/thaisum/thaisum/1.0.0/347b33c852af4d796e1224e00e15142d626608a9fa3e07ad6d19dfd8fcae5423)


In [9]:
train_data = train_data+ list(thaisum['train'].data['body']) +list(thaisum['test'].data['body'])

In [12]:
print(len(train_data))

431036


In [10]:
del thaisum

**thai_toxicity_tweet**

In [11]:
thai_toxicity_tweet_dataset = load_dataset("thai_toxicity_tweet")

Reusing dataset thai_toxicity_tweet (/root/.cache/huggingface/datasets/thai_toxicity_tweet/thai_toxicity_tweet/1.0.0/6156251739b83a8e57f9ed4adf0f1d134e9bfded6d2870647558fdb7a0e60306)


In [14]:
len(thai_toxicity_tweet_dataset['train']['tweet_text'])

3300

In [15]:
_t=list([i for i in thai_toxicity_tweet_dataset['train']['tweet_text'] if i!="TWEET_NOT_FOUND"])

In [16]:
len(_t)

2794

In [19]:
train_data = train_data+ _t

In [20]:
del thai_toxicity_tweet_dataset,_t

In [21]:
print(len(train_data))

433830


**wongnai reviews**

In [22]:
wongnai_reviews_dataset = load_dataset("wongnai_reviews")

Using custom data configuration default
Reusing dataset wongnai_reviews (/root/.cache/huggingface/datasets/wongnai_reviews/default/1.0.1/0741f812b51dea635d957a5cc8f46fcc98828c0f6c557336ab20367778299a4d)


In [23]:
wongnai_reviews_dataset

DatasetDict({
    train: Dataset({
        features: ['review_body', 'star_rating'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['review_body', 'star_rating'],
        num_rows: 6203
    })
})

In [24]:
train_data = train_data+ list(wongnai_reviews_dataset['train']['review_body'])+list(wongnai_reviews_dataset['test']['review_body'])

In [25]:
print(len(train_data))

480033


**wisesight_sentiment**

In [26]:
wisesight_sentiment_dataset= load_dataset("wisesight_sentiment")

Reusing dataset wisesight_sentiment (/root/.cache/huggingface/datasets/wisesight_sentiment/wisesight_sentiment/1.0.0/fc2b1bdfe79571b2e281e4afdb5aac069cf9270bf0f85694239be672a4191969)


In [27]:
train_data += list(wisesight_sentiment_dataset['train']['texts'])+list(wisesight_sentiment_dataset['test']['texts'])

In [28]:
print(len(train_data))

504332


In [29]:
del wisesight_sentiment_dataset

**TLC**

In [30]:
tlc_dataset = load_dataset("tlc")

No config specified, defaulting to: tlc/tlcv2.0
Reusing dataset tlc (/root/.cache/huggingface/datasets/tlc/tlcv2.0/2.0.0/d29676ddf936f9802990caebd5eafc8e960d51e9e02b86d0baf49b634587d2be)


In [31]:
train_data += list(tlc_dataset['train']['text'][0][0])

In [32]:
print(len(train_data))

504534


In [33]:
del tlc_dataset

**scb_mt_enth_2020**

In [34]:
scb_mt_enth_dataset = load_dataset("scb_mt_enth_2020", 'then')

Reusing dataset scb_mt_enth2020 (/root/.cache/huggingface/datasets/scb_mt_enth2020/then/1.0.0/3bb5a43042a84446b254e215a8b1604b454ad56ada3370309a6f6a3b480c86a8)


In [35]:
th=[i['th'] for i in scb_mt_enth_dataset['train']['translation']]+[i['th'] for i in scb_mt_enth_dataset['test']['translation']]

In [36]:
len(th)

901579

In [37]:
train_data +=th

In [38]:
del th

In [39]:
print(len(train_data))

1406113


**Thai Wiki**

In [40]:
with open('wiki_20210620.txt', 'r', encoding='utf-8-sig') as f:
    train_data+=[i.strip().replace('()','').replace('( )','').replace('(; ;)','').replace('(; ; )','').replace('(; )','').replace('(, )','') for i in f.readlines()]

In [41]:
print(len(train_data))

2718097


In [34]:
# def clean_data(text):
#     train_data_clean.append(normalize(str(text).strip().replace('</s>','')))

In [42]:
from tqdm import tqdm

In [43]:
train_data = [str(i).strip().replace('</s>','') for i in tqdm(train_data)]

100%|██████████| 2718097/2718097 [00:12<00:00, 224433.53it/s]


In [44]:
t=[i for i in train_data if len(i)<=280 and len(i)>=10]

In [45]:
len(t)

1697254

In [46]:
len(t)/len(train_data)*100

62.44273107251139

In [82]:
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPTNeoForCausalLM

In [83]:
torch.manual_seed(42)

In [101]:
pretraine = "EleutherAI/gpt-neo-125M"

In [102]:
device=torch.device("cuda:1")

In [103]:
tokenizer = GPT2Tokenizer.from_pretrained(pretraine, bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPTNeoForCausalLM.from_pretrained(pretraine).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [104]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50258, 768)

In [53]:
max_length = 280#240

In [54]:
class TestDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [55]:
dataset = TestDataset(t, tokenizer, max_length=max_length)

In [56]:
train_size = int(0.90 * len(dataset))

In [57]:
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

In [58]:
len(train_dataset),len(val_dataset)

(1527528, 169726)

In [59]:
len(dataset)

1697254

In [71]:
tokenizer.decode(dataset[1009][0], skip_special_tokens=True)

'<|startoftext|>เจ็บใจฟุตบอลหญิง ถ้าเราได้คือแบบช่างแม่งเหอะเจ้าเหรียญห่าเหวไรที่โกงๆเนี๊ย แต่แม่งโกงบอลหญิงเราไป บอลชายล้างแค้นให้เด้อ #บอลไทย #ซีเกมส์2017'

In [105]:
training_args = TrainingArguments(
    output_dir='./results',
    do_train=True,
    do_eval=False,
    evaluation_strategy="epoch",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    logging_strategy="epoch",
    save_strategy="steps",
    per_device_eval_batch_size=2,
    warmup_steps=100,
    logging_steps=1000,
    weight_decay=0.01,
    logging_dir='./logs'
)

In [106]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=val_dataset,
    data_collator=lambda data: {
        'input_ids': torch.stack([f[0] for f in data]),
        'attention_mask': torch.stack([f[1] for f in data]),
        'labels': torch.stack([f[0] for f in data])
    }
)

In [107]:
trainer.train("results/checkpoint-212000").to(device)

/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
2,0.285500,0.276673


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



AttributeError: 'TrainOutput' object has no attribute 'to'

In [108]:
model.save_pretrained('./thaigpt-next-125m')

In [109]:
tokenizer.save_pretrained("./thaigpt-next-125m/tokenizer/")

('./thaigpt-next-125m/tokenizer/tokenizer_config.json',
 './thaigpt-next-125m/tokenizer/special_tokens_map.json',
 './thaigpt-next-125m/tokenizer/vocab.json',
 './thaigpt-next-125m/tokenizer/merges.txt',
 './thaigpt-next-125m/tokenizer/added_tokens.json')

In [111]:
import pickle

In [112]:
with open('dataset.pickle', 'wb') as handle:
    pickle.dump(dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [113]:
with open('dataset.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [114]:
dataset==b

False

In [115]:
b

In [116]:
dataset